# Create a single file with all HYCOM and forcing data together, for all the selected events, in all basins, with HYCOM data interpolated at hourly resolution

In [2]:
import xarray as xr
import csv
import pandas as pd
import pickle
import numpy as np
import os
from datetime import date, timedelta, datetime
import sys
import matplotlib.pyplot as plt
import netCDF4 as nc4
import glob
import os.path
import webbrowser
import time
# from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import matplotlib as mpl
from FUNCTIONS_HYCOM import set_regions, set_regions_tags, find_timedelta64_index
import pickle as pkl
import subprocess
# import metpy
# from metpy.calc import relative_humidity_from_specific_humidity
# from metpy.units import units
import xarray as xr
# import metpy.calc as mpcalc
# from metpy.units import units


In [4]:
relative_humidity_tag = True
precip_tag = False


In [46]:
# Function to add hourly forcing data to HYCOM data, which were previously interpolated at hourly resolution
def HYCOM_forcing_merge(bfr_combined_ocean, bfr_combined_ocean_index_sel, HYCOM_forcing_dir, folder_tag, forcing_filename):
    # Initialize list to store forcing data
    bfr_forcing_file = []
    
    # Loop through all dates needed for the selected event
    for idate in np.unique(bfr_combined_ocean.sel(index=bfr_combined_ocean_index_sel[0]).time_event_original.values.astype('datetime64[D]')):
        # If idate is in 2010: forcing does not exist, set NaNs
        if pd.to_datetime(idate).year < 2011:
            # Open a forcing file from one day in 2011 (10 days after the 2010 date) to make sure the file structure stays the same
            forcing_temp_2010 = xr.open_dataset(HYCOM_forcing_dir + folder_tag + '/' + \
                                               folder_tag + '_' + forcing_filename + '_' + \
                                               pd.to_datetime(idate + np.timedelta64(10,'D')).strftime('%Y%m%d') + \
                                               '.nc4', engine='netcdf4')
            # Change the time vector to the desired day in 2010 (go back 10 days)
            forcing_temp_2010['MT'] = forcing_temp_2010['MT'] - np.timedelta64(10,'D')
            # Set all forcing variables to NaNs
            for var_name in forcing_temp_2010.data_vars:
                forcing_temp_2010[var_name].loc[:] = np.nan
            # Append to list
            bfr_forcing_file.append(forcing_temp_2010)
        else: # if 2011 or later, forcing exists, so open relevant dates and put all data in one list
            bfr_forcing_file.append(xr.open_dataset(HYCOM_forcing_dir + folder_tag + '/' + \
                                               folder_tag + '_' + forcing_filename + '_' + \
                                               pd.to_datetime(idate).strftime('%Y%m%d') + \
                                               '.nc4', engine='netcdf4'))

    # Concatenate all forcing into a single xarray
    bfr_forcing_file_dxr = xr.concat(bfr_forcing_file, dim='MT')

    # Re-project longitude to match HYCOM longitude values
    lon_new = bfr_forcing_file_dxr.Longitude.values
    lon_new[lon_new>180] = lon_new[lon_new>180] - 360
    bfr_forcing_file_dxr = bfr_forcing_file_dxr.assign_coords(Longitude=lon_new)

    bfr_diff_list = abs(bfr_forcing_file_dxr.MT.values - bfr_combined_ocean.sel(index=bfr_combined_ocean_index_sel[0])\
                        .time_event_original.values[0]).tolist()
    bfr_ind_start = bfr_diff_list.index(min(bfr_diff_list))
    bfr_diff_list = abs(bfr_forcing_file_dxr.MT.values - bfr_combined_ocean.sel(index=bfr_combined_ocean_index_sel[0])\
                        .time_event_original.values[-1]).tolist()
    bfr_ind_end = bfr_diff_list.index(min(bfr_diff_list))

    bfr_forcing_file_dxr_sel = bfr_forcing_file_dxr.isel(MT = slice(bfr_ind_start, bfr_ind_end+1))

    for iidx in bfr_combined_ocean_index_sel:
        bfr_forcing_file_dxr_sel_interp = bfr_forcing_file_dxr_sel.interp(Longitude = bfr_combined_ocean.sel(index=iidx).longitude.values, \
                                    Latitude = bfr_combined_ocean.sel(index=iidx).latitude.values)
        
        bfr_forcing_file_dxr_sel_interp = bfr_forcing_file_dxr_sel_interp.assign_coords(MT=bfr_combined_ocean.sel(index=bfr_combined_ocean_index_sel[0]).delta_time.values)
        bfr_forcing_file_dxr_sel_interp = bfr_forcing_file_dxr_sel_interp.assign_coords(index=iidx)
        bfr_forcing_file_dxr_sel_interp = bfr_forcing_file_dxr_sel_interp.expand_dims('index')

        bfr_forcing_file_dxr_sel_interp = bfr_forcing_file_dxr_sel_interp.rename_dims({'MT': 'delta_time'})
        bfr_forcing_file_dxr_sel_interp = bfr_forcing_file_dxr_sel_interp.rename({'MT': 'delta_time'})
        bfr_forcing_file_dxr_sel_interp = bfr_forcing_file_dxr_sel_interp.drop('Longitude')
        bfr_forcing_file_dxr_sel_interp = bfr_forcing_file_dxr_sel_interp.drop('Latitude')

        bfr_combined_ocean = xr.merge([bfr_combined_ocean, bfr_forcing_file_dxr_sel_interp]) 
        
    return(bfr_combined_ocean)


In [4]:
# MAIN PART

# List all files across basins
folder_tags = ['NorthAtlantic', 'EPacific', 'WPacific_1of3', 'WPacific_2of3', 'WPacific_3of3', 'SWPacific_1of2', 'SWPacific_2of2', 'Indian']
HYCOM_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/6_HYCOM_near_TC_53x_pkl_NEW_BOX_postproc_/'
HYCOM_forcing_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/HYCOM_FORCING/'
number_of_days_before_event = 6
number_of_days_after_event = 24
forcing_filenames = ['TaqaQrQp','dswsfc','precip','sfcprs','surtmp','uv-10m','wndspd', 'dlwsfc']

# Add info from the rest
datasets = []
index = [] 
N = 0

# Loop through regions
for i in np.arange(len(folder_tags)):
    
    # List files for this region
    HYCOM_dir_current = HYCOM_dir.replace('postproc_', 'postproc_' + str(folder_tags[i])) 
    files = [j for j in sorted(glob.glob(HYCOM_dir_current + '*MLD.pkl'))]
        
    # Loop through all the files of a single region
    N_start = N
    for ifile in np.arange(len(files)):
        
        # Load HYCOM file
        df = pkl.load(open(files[ifile], "rb"))

        # Remove potential duplicate times
        _, index = np.unique(df['time'], return_index=True)
        df_unique_time = df.isel(time=index)
        
        # Find range of days we care about
        date_of_event = pd.to_datetime(df.sel_date.values, format='%Y-%m-%d %H:%M:%S').to_datetime64()
        start_date_event = date_of_event - np.timedelta64(number_of_days_before_event, 'D')
        end_date_event = date_of_event + np.timedelta64(number_of_days_after_event, 'D')

        # Create new time axis (hourly resolution)
        time_new = pd.date_range(start_date_event, end_date_event, freq='1h')

        # Interpolate data using new time axis just created
        df_unique_time_interp = df_unique_time.interp(time=time_new)

        df_unique_time_interp["index_tag"] = files[ifile].split("/")[-1].split("event")[0] + 'event_' + \
        files[ifile].split("/")[-1].split("event")[-1].split("_")[1]

        df_unique_time_interp['time_event_original'] = df_unique_time_interp.time
        
        # Save date_of_event
        df_unique_time_interp['date_of_event'] = date_of_event

        df_unique_time_interp = df_unique_time_interp.assign_coords(time=df_unique_time_interp.time.values - date_of_event)
        df_unique_time_interp = df_unique_time_interp.assign_coords(index=N)
        df_unique_time_interp = df_unique_time_interp.expand_dims('index')

        df_unique_time_interp = df_unique_time_interp.rename_dims({'time': 'delta_time'})
        df_unique_time_interp = df_unique_time_interp.rename({'time': 'delta_time'})
        
        # Append to create a single macro-list of all events (hourly res) for the region being considered
        datasets.append(df_unique_time_interp)
        
        # Count events (across all regions)
        N = N+1
    
    # Concatenate all events in the region in a single xarray
    bfr_combined = xr.concat(datasets[N_start:N], dim="index")
    # List all event dates for this region
    bfr_date_event_unique = np.unique(bfr_combined.date_of_event) 
        
    # Loop across all event dates
    for itime in bfr_date_event_unique:
        # Grab data from each event
        bfr_combined_index_sel = bfr_combined.where(bfr_combined.date_of_event==itime, drop=True).squeeze().index.values.tolist()
        # Find index of current event
        if type(bfr_combined_index_sel)==int:
            bfr_combined_index_sel = [bfr_combined_index_sel]
        elif type(bfr_combined_index_sel)!=list:
            print('Error list')
            ciao

        # Loop acoss all types of forcing files
        for ifilename in forcing_filenames:
            # Add forcing data to HYCOM hourly data for selected event
            bfr_combined = HYCOM_forcing_merge(bfr_combined_ocean = bfr_combined, 
                                               bfr_combined_ocean_index_sel = bfr_combined_index_sel,
                                               HYCOM_forcing_dir = HYCOM_forcing_dir, 
                                               folder_tag = folder_tags[i], 
                                               forcing_filename = ifilename)

    # Combine all bfr_combined together
    if i==0:
        combined_regions = bfr_combined
    else:
        combined_regions = xr.merge([combined_regions, bfr_combined])

# Combine HYCOM data only for all regions
combined = xr.concat(datasets, dim='index')

# Save HYCOM data as netcdf
combined.to_netcdf('combined.nc4')
# Save HYCOM + forcing data as netcdf
combined_regions.to_netcdf('combined_regions.nc4')



NameError: name 'ciao' is not defined

In [ ]:
ciao


### Compute relative humidity

In [3]:
if relative_humidity_tag:
    # Load the xarray dataset
    # Replace 'your_dataset_path' with the actual path to your dataset file
    ds = xr.open_dataset('/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/combined_regions.nc4')

    # Define scale factor and offset for air pressure conversion
    add_offset = -31.413218 # Your offset
    scale_factor = 0.00390625 # Your scale factor

    # Calculate adjusted air pressure
    adjusted_airprs = ((ds['airprs'] * scale_factor + add_offset) + 1000) * units.hPa

    # Select the required variables
    airtime = ds['airtmp']
    vapmix = ds['vapmix']

    # Calculate relative humidity
    relative_humidity = mpcalc.relative_humidity_from_specific_humidity(
        adjusted_airprs, airtime, vapmix
    )

    # Calculate percent relative humidity
    percent_relative_humidity = relative_humidity * 100

    # Print or use the resulting percent relative humidity
#     print(percent_relative_humidity)

    # Add the new variable to the original dataset
    ds['percent_relative_humidity'] = percent_relative_humidity
    ds['adjusted_airprs'] = adjusted_airprs
    # Save the modified dataset if needed
    ds.to_netcdf('combined_regions_with_rel_hum.nc4')


In [45]:
ds = xr.open_dataset('/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/combined_regions.nc4')

ds

<xarray.Dataset>
Dimensions:                  (index: 1238, delta_time: 721, depth: 28)
Coordinates:
  * index                    (index) int64 0 1 2 3 4 ... 1234 1235 1236 1237
  * depth                    (depth) float64 0.0 2.0 4.0 ... 350.0 400.0 500.0
    longitude                (index) float64 ...
    latitude                 (index) float64 ...
    integral_precip          (index) float64 ...
    sel_date                 (index) object ...
  * delta_time               (delta_time) timedelta64[ns] -6 days +00:00:00 ....
Data variables: (12/33)
    temperature_abs          (index, delta_time, depth) float64 ...
    salinity_abs             (index, delta_time, depth) float64 ...
    density_abs              (index, delta_time, depth) float64 ...
    temperature_anom         (index, delta_time, depth) float64 ...
    salinity_anom            (index, delta_time, depth) float64 ...
    density_anom             (index, delta_time, depth) float64 ...
    ...                       ...
    airprs                   (index, delta_time) float64 ...
    surtmp                   (index, delta_time) float64 ...
    wndnwd                   (index, delta_time) float64 ...
    wndewd                   (index, delta_time) float64 ...
    wndspd                   (index, delta_time) float64 ...
    dlwflx                   (index, delta_time) float64 ...
Attributes:
    description:  HYCOM T/S co-located with TCs

In [ ]:
# if precip_tag:
#     # Load the xarray dataset
#     # Replace 'your_dataset_path' with the actual path to your dataset file
#     ds = xr.open_dataset('/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/combined_regions.nc4')

#     # Define scale factor and offset for air pressure conversion
#     add_offset = 0.03125 # Your offset
#     scale_factor = 9.536743e-07 # Your scale factor

#     # Calculate adjusted air pressure
#     adjusted_precip = ((ds['precip'] * scale_factor + add_offset))

    

In [ ]:
test = nc4.Dataset('/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/combined_regions.nc4')
precip = test.variables['precip']


In [ ]:
precip

In [12]:
# t = xr.open_dataset('/Users/jacoposala/Downloads/netcdf_cfsv2-sea_2020_01hr_precip.nc4')
# t = xr.open_dataset('/Users/jacoposala/Downloads/netcdf_cfsv2-sea_2020_01hr_precip.nc4')
# t = xr.open_dataset('/Users/jacoposala/Downloads/netcdf_cfsv2-sec2_2019_01hr_uv-10m.nc4')
test = nc4.Dataset('/Users/jacoposala/Downloads/netcdf_cfsv2-sec_2014_01hr_sfcprs.nc4')


In [13]:
test

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    title: CFSV2 0.205g 2014 1h sfcprs sm10diff
    institution: Naval Research Laboratory
    source: NRL .D forcing file
    history: Mon Apr 27 14:12:30 2015: /site/applic/nco/4.2.1-intel/bin/ncatted -O -a units,airprs,o,c,hPa /p/cwfs/dsfrank/force/cfsv2/0.205g/1hrly/process/output/2014/cfsv2-sec_2014_01hr_sfcprs.nc
nrl2nc
    History: Translated to CF-1.0 Conventions by Netcdf-Java CDM (CFGridWriter2)
Original Dataset = /hycom/ftp/datasets/force/ncep_cfsv2/netcdf/cfsv2-sec_2014_01hr_sfcprs.nc; Translation Date = 2023-08-28T16:11:14.655Z
    geospatial_lat_min: -89.84351351786847
    geospatial_lat_max: 89.84351351786847
    geospatial_lon_min: 0.0
    geospatial_lon_max: 359.7954521626234
    dimensions(sizes): MT(1), Latitude(880), Longitude(1760)
    variables(dimensions): int16 airprs(MT, Latitude, Longitude), float64 MT(MT), float64 Latitude(Latitude), float64 Longitude

In [ ]:
nc4 = nc4.Dataset('/Users/jacoposala/Downloads/netcdf_cfsv2-sea_2020_01hr_precip.nc4')

In [ ]:
nc4.variables['precip']

In [ ]:
t.precip

In [ ]:
t['precip']

In [ ]:
adjusted_precip[0,1,1]

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Load the xarray dataset
t = xr.open_dataset('/Users/jacoposala/Downloads/netcdf_cfsv2-sea_2020_01hr_precip.nc4')

# Define scale factor and offset for air pressure conversion
add_offset = 0.03125 # Your offset
scale_factor = 9.536743e-07 # Your scale factor

# Calculate adjusted air pressure
adjusted_precip = ((t['precip'] * scale_factor) + add_offset)
    
precip_data = adjusted_precip
# Extract data variables
# precip_data = t['precip']

# Choose a specific time step (MT index), you can modify this accordingly
time_index = 0

# Extract precipitation data for the chosen time step
precip_time_step = precip_data.isel(MT=time_index)

# Get latitude and longitude values
lats = precip_data['Latitude'].values
lons = precip_data['Longitude'].values

# Create a map plot using Matplotlib and Cartopy
plt.figure(figsize=(20, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

# Plot the precipitation data using pcolormesh
plt.pcolormesh(lons, lats, precip_time_step, cmap='rainbow')

# Add colorbar
cbar = plt.colorbar()
cbar.set_label('Precipitation rate (kg m-2 s-1)')

# Set plot title and labels
# plt.title('Precipitation Map')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the plot
plt.show()


In [ ]:
# # LOAD high-speed event information (lat, lon, time)

# # List all files across basins
# # folder_tags = ['WPacific_3of3', 'NorthAtlantic']
# folder_tags = ['NorthAtlantic', 'EPacific', 'WPacific_1of3', 'WPacific_2of3', 'WPacific_3of3', 'SWPacific_1of2', 'SWPacific_2of2', 'Indian']
# HYCOM_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/6_HYCOM_near_TC_53x_pkl_NEW_BOX_postproc_/'
# HYCOM_forcing_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/HYCOM_FORCING/REGIONS/'
# # timesteps_per_day = 8
# number_of_days_before_event = 6
# number_of_days_after_event = 24

# forcing_filenames = ['TaqaQrQp']


# # Add info from the rest
# datasets = []
# index = [] 
# N = 0
# # files = []

# #Loop through regions
# for i in np.arange(len(folder_tags)):

#     HYCOM_dir_current = HYCOM_dir.replace('postproc_', 'postproc_' + str(folder_tags[i])) 
#     files = [j for j in sorted(glob.glob(HYCOM_dir_current + '*MLD.pkl'))]
    
#     N_start = N
#     # Loop through all the files of a single region
#     for ifile in np.arange(len(files)):
#         df = pkl.load(open(files[ifile], "rb"))
#         # 
#         _, index = np.unique(df['time'], return_index=True)
#         df_unique_time = df.isel(time=index)
#         date_of_event = pd.to_datetime(df.sel_date.values, format='%Y-%m-%d %H:%M:%S').to_datetime64()
#         start_date_event = date_of_event - np.timedelta64(number_of_days_before_event, 'D')
#         end_date_event = date_of_event + np.timedelta64(number_of_days_after_event, 'D')

#         # Create new time axis (hourly)
#         time_new = pd.date_range(start_date_event, end_date_event, freq='1h')

#     #         time_new = pd.date_range(df.time.values[0], df.time.values[0] + np.timedelta64(int(len(df.time.values)/timesteps_per_day), 'D'), freq='1h')
#         # Interpolation using new time axis just created
#         df_unique_time_interp = df_unique_time.interp(time=time_new)
#     #         if len(df_unique_time_interp.time.values):
#     #             print('Check what is wrong')
#     #             ciao1
#     #         print(df.time.values)
#     #         df['time'] = time_new


#         df_unique_time_interp["index_tag"] = files[ifile].split("/")[-1].split("event")[0] + 'event_' + \
#         files[ifile].split("/")[-1].split("event")[-1].split("_")[1]

#         df_unique_time_interp['time_event_original'] = df_unique_time_interp.time
#         # save date_of_event
#         df_unique_time_interp['date_of_event'] = date_of_event

#         df_unique_time_interp = df_unique_time_interp.assign_coords(time=df_unique_time_interp.time.values - date_of_event)
#         df_unique_time_interp = df_unique_time_interp.assign_coords(index=N)
#         df_unique_time_interp = df_unique_time_interp.expand_dims('index')

#         df_unique_time_interp = df_unique_time_interp.rename_dims({'time': 'delta_time'})
#         df_unique_time_interp = df_unique_time_interp.rename({'time': 'delta_time'})
        
#         datasets.append(df_unique_time_interp)
#         N = N+1
#     bfr_combined = xr.concat(datasets[N_start:N], dim="index")
#     bfr_date_event_unique = np.unique(bfr_combined.date_of_event) 
    
#     for itime in bfr_date_event_unique[0:2]:
#         bfr_combined_index_sel = bfr_combined.where(bfr_combined.date_of_event==itime, drop=True).squeeze().index.values.tolist()
#         if type(bfr_combined_index_sel)==int:
#             bfr_combined_index_sel = [bfr_combined_index_sel]
#         elif type(bfr_combined_index_sel)!=list:
#             print('Error list')
#             ciao
#         # Loop through indices
# #         bfr_time_orig_sel = []
# #         for iidx in bfr_combined_index_sel:
# #             bfr_time_orig_sel.append(bfr_combined.isel(index=iidx).time_event_original.values)
    
# #         bfr_time_orig_sel_unique = np.unique(np.vstack(bfr_time_orig_sel).flatten())
#         for ifilename in forcing_filenames:
        
    
#             bfr_combined = HYCOM_forcing_merge(bfr_combined_ocean = bfr_combined, bfr_combined_ocean_index_sel = bfr_combined_index_sel, \
#                         HYCOM_forcing_dir = HYCOM_forcing_dir, folder_tag = folder_tags[i], forcing_filename = ifilename)

#     if i==0:
#         combined_regions = bfr_combined
#     else:
#         combined_regions = xr.merge([combined_regions, bfr_combined])
        
        
        
# #     print('1+++++++++++++++++++++++++++++++')
# #     print(len(datasets))    
# #     datasets = datasets[0:N_start]
# #     print('2++++++++++++++++++++++++++++++++++')
# #     print(len(datasets))     
# #     datasets.append(bfr_combined)
# #     print('3++++++++++++++++++++++++++++++++')
# #     print(len(datasets)) 

    
#     #             ciao
        
        
# #         bfr_TaqaQrQp = []
# #         for idate in np.unique(bfr_combined.isel(index=bfr_combined_index_sel[0]).time_event_original.values.astype('datetime64[D]')):
# #             bfr_TaqaQrQp.append(xr.open_dataset(HYCOM_forcing_dir + folder_tags[i] + '/' + \
# #                                                folder_tags[i] + '_TaqaQrQp_' + \
# #                                                pd.to_datetime(idate).strftime('%Y%m%d') + \
# #                                                '.nc4', engine='netcdf4'))
            
# #         bfr_TaqaQrQp_dxr = xr.concat(bfr_TaqaQrQp, dim='MT')
        
# #         lon_new = bfr_TaqaQrQp_dxr.Longitude.values
# #         lon_new[lon_new>180] = lon_new[lon_new>180] - 360
# #         bfr_TaqaQrQp_dxr = bfr_TaqaQrQp_dxr.assign_coords(Longitude=lon_new)
        
# #         bfr_diff_list = abs(bfr_TaqaQrQp_dxr.MT.values - bfr_combined.isel(index=bfr_combined_index_sel[0])\
# #                             .time_event_original.values[0]).tolist()
# #         bfr_ind_start = bfr_diff_list.index(min(bfr_diff_list))
# #         bfr_diff_list = abs(bfr_TaqaQrQp_dxr.MT.values - bfr_combined.isel(index=bfr_combined_index_sel[0])\
# #                             .time_event_original.values[-1]).tolist()
# #         bfr_ind_end = bfr_diff_list.index(min(bfr_diff_list))
        
# #         bfr_TaqaQrQp_dxr_sel = bfr_TaqaQrQp_dxr.isel(MT = slice(bfr_ind_start, bfr_ind_end+1))
        
        
# #         for iidx in bfr_combined_index_sel:
# #             bfr_TaqaQrQp_dxr_sel_interp = bfr_TaqaQrQp_dxr_sel.interp(Longitude = bfr_combined.isel(index=iidx).longitude.values, \
# #                                         Latitude = bfr_combined.isel(index=iidx).latitude.values)
            
            
# #             bfr_TaqaQrQp_dxr_sel_interp = bfr_TaqaQrQp_dxr_sel_interp.assign_coords(MT=bfr_combined.isel(index=bfr_combined_index_sel[0]).delta_time.values)
# #             bfr_TaqaQrQp_dxr_sel_interp = bfr_TaqaQrQp_dxr_sel_interp.assign_coords(index=iidx)
# #             bfr_TaqaQrQp_dxr_sel_interp = bfr_TaqaQrQp_dxr_sel_interp.expand_dims('index')

# #             bfr_TaqaQrQp_dxr_sel_interp = bfr_TaqaQrQp_dxr_sel_interp.rename_dims({'MT': 'delta_time'})
# #             bfr_TaqaQrQp_dxr_sel_interp = bfr_TaqaQrQp_dxr_sel_interp.rename({'MT': 'delta_time'})
        
        
            
# #             bfr_combined = xr.merge([bfr_combined, bfr_TaqaQrQp_dxr_sel_interp]).delta_time   
            
# #             ciao
# #         if len(bfr_combined_index_sel)>1:
# #                 print('stop')
# #                 ciao
# #     ciao
# #     # Open all forcing files for this date
# #     forcing_TaqaQrQp = xr.open_dataset(HYCOM_forcing_dir + folder_tags[i] + '/' + folder_tags[i] + '_TaqaQrQp_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')
# #     forcing_precip = xr.open_dataset(HYCOM_forcing_dir + folder_tags[i] + '/' + folder_tags[i] + '_precip_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')
# #     forcing_surtmp = xr.open_dataset(HYCOM_forcing_dir + folder_tags[i] + '/' + folder_tags[i] + '_surtmp_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')
# # #   forcing_dswsfc = xr.open_dataset(HYCOM_forcing_dir + folder_tags[i] + '/' + folder_tags[i] + '_dswsfc_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')
# #     forcing_uv_10m = xr.open_dataset(HYCOM_forcing_dir + folder_tags[i] + '/' + folder_tags[i] + '_uv-10m_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')
# #     forcing_wndspd = xr.open_dataset(HYCOM_forcing_dir + folder_tags[i] + '/' + folder_tags[i] + '_wndspd_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')
# # #   forcing_sfcprs = xr.open_dataset(HYCOM_forcing_dir + folder_tags[i] + '/' + folder_tags[i] + '_sfcprs_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')

    


# # print('before combined') 

# combined = xr.concat(datasets, dim='index')

# # Save netcdf
# combined.to_netcdf('combined.nc4')


In [ ]:
# Open netcdf
# xr.open_dataset('combined.nc4')

In [ ]:
# Check: if the minimum it is not True: PROBLEM 
# min(np.abs(combined.time_event_original.isel(delta_time=6*24).values - combined.date_of_event.values)) == 0

In [ ]:
# # Loop across indexes
# for i in np.arange(int(combined.index[-1].values)):

#     # Find region to consider for forcing files
#     sel_region = str(combined.isel(index = i).index_tag.values)[0:-7] # if the event number has 3 digits, we this is what we need
#     while sel_region[-1] != '_': # if the event number has less than 3 digits, we need to remove some other characters at the end
#         sel_region = sel_region[0:-1]
#     sel_region = sel_region[0:-1] # remove the last underscore to match the folder name for the forcing file

#     # Find range of dates to open in forcing files
#     start_date = combined.isel(index = i).time_event_original[0].values
#     end_date = combined.isel(index = i).time_event_original[-1].values
#     range_dates = pd.date_range(pd.to_datetime(start_date),pd.to_datetime(end_date), freq = 'd')
    
#     # Find desired lat/lon
#     sel_lat = float(combined.isel(index = i).latitude.values)
#     sel_lon = float(combined.isel(index = i).longitude.values)

#     # Initialize variables to store data
#     airtmp = []
#     vapmix = []
#     precip = []
#     radflx = []
#     shwflx = []
#     surtmp = []
#     dlwflx = []
#     dswflx = []
#     wndnwd = []
#     wndewd = []
#     wndspd = []
#     sfcprs = []
    
#     # Loop across range of dates to open forcing files
#     for sel_date in range_dates:
#         # Open all forcing files for this date
#         forcing_TaqaQrQp = xr.open_dataset(HYCOM_forcing_dir + sel_region + '/' + sel_region + '_TaqaQrQp_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')
#         forcing_precip = xr.open_dataset(HYCOM_forcing_dir + sel_region + '/' + sel_region + '_precip_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')
#         forcing_surtmp = xr.open_dataset(HYCOM_forcing_dir + sel_region + '/' + sel_region + '_surtmp_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')
# #         forcing_dswsfc = xr.open_dataset(HYCOM_forcing_dir + sel_region + '/' + sel_region + '_dswsfc_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')
#         forcing_uv_10m = xr.open_dataset(HYCOM_forcing_dir + sel_region + '/' + sel_region + '_uv-10m_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')
#         forcing_wndspd = xr.open_dataset(HYCOM_forcing_dir + sel_region + '/' + sel_region + '_wndspd_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')
# #         forcing_sfcprs = xr.open_dataset(HYCOM_forcing_dir + sel_region + '/' + sel_region + '_sfcprs_' + pd.to_datetime(sel_date).strftime('%Y%m%d') + '.nc4', engine='netcdf4')
        
#         # Find lat/lon indexes to save
#         lat_index = (np.abs(forcing_uv_10m.Latitude.values - sel_lat)).argmin()
#         lon_index = (np.abs(forcing_uv_10m.Longitude.values - sel_lon)).argmin()
        
#         # Find time indexes to save
#         if sel_date == range_dates[0]: # first date in range
#             start_time_index = sel_date.hour
#             end_time_index = 24
            
#         elif sel_date == range_dates[-1]: # last date in range
#             start_time_index = 0
#             end_time_index = sel_date.hour + 1
            
#         else: # all dates in between
#             start_time_index = 0
#             end_time_index = 24
    
#         airtmp.append(forcing_TaqaQrQp.airtmp[start_time_index:end_time_index,lat_index,lon_index].values)
#         vapmix.append(forcing_TaqaQrQp.vapmix[start_time_index:end_time_index,lat_index,lon_index].values)
#         radflx.append(forcing_TaqaQrQp.radflx[start_time_index:end_time_index,lat_index,lon_index].values)
#         shwflx.append(forcing_TaqaQrQp.shwflx[start_time_index:end_time_index,lat_index,lon_index].values)
#         precip.append(forcing_precip.precip[start_time_index:end_time_index,lat_index,lon_index].values)        
#         surtmp.append(forcing_surtmp.surtmp[start_time_index:end_time_index,lat_index,lon_index].values)      
# #         dswflx.append(forcing_dswsfc.dswflx[start_time_index:end_time_index,lat_index,lon_index].values)
# #         dlwflx.append(forcing_dswsfc.dlwflx[start_time_index:end_time_index,lat_index,lon_index].values)
#         wndnwd.append(forcing_uv_10m.wndnwd[start_time_index:end_time_index,lat_index,lon_index].values)
#         wndewd.append(forcing_uv_10m.wndewd[start_time_index:end_time_index,lat_index,lon_index].values)
#         wndspd.append(forcing_wndspd.wndspd[start_time_index:end_time_index,lat_index,lon_index].values)
# #         sfcprs.append(forcing_sfcprs.sfcprs[start_time_index:end_time_index,lat_index,lon_index].values)

#     # Concatenate list -- results for this event
#     airtmp = np.concatenate(airtmp)
#     vapmix = np.concatenate(vapmix)
#     radflx = np.concatenate(radflx)
#     shwflx = np.concatenate(shwflx)
#     precip = np.concatenate(precip)
#     surtmp = np.concatenate(surtmp)
#     #dswflx = np.concatenate(dswflx)
#     #dlwflx = np.concatenate(dlwflx)
#     wndnwd = np.concatenate(wndnwd)    
#     wndewd = np.concatenate(wndewd)   
#     wndspd = np.concatenate(wndspd)   
#     #sfcprs = np.concatenate(sfcprs)   

#     ciao        
        
        
    